In [10]:
from pathlib import Path
import pandas as pd
import re
import numpy as np

In [3]:
msi_csv = "/storage/scratch1/2/yhao306/sma-brain/data-sm/V11L12-038_A1_MSI_raw_counts_spotRows.csv"
msi_df = pd.read_csv(msi_csv, index_col=0)
coord_csv = "/storage/scratch1/2/yhao306/sma-brain/data-sm/V11L12-038_A1_spot_coordinates.csv"
coords_df = pd.read_csv(coord_csv, index_col=0) 
msi_csv = "../msi_norm.csv"
msi_norm = pd.read_csv(msi_csv, index_col=0)
msi_norm_hvg = "../msi_norm_hvg.csv"
msi_norm_hvg = pd.read_csv(msi_norm_hvg, index_col=0)
msi_csv_hvg = "../msi_df_hvg.csv"
msi_df_hvg = pd.read_csv(msi_csv_hvg, index_col=0)
rna_norm_hvg = "../rna_norm_hvg.csv"
rna_norm_hvg = pd.read_csv(rna_norm_hvg, index_col=0)
rna_csv = "../rna_norm.csv"
rna_norm = pd.read_csv(rna_csv, index_col=0)
region_csv = "/storage/scratch1/2/yhao306/sma-brain/data-sm/sma/V11L12-038/V11L12-038_A1/output_data/V11L12-038_A1_RNA/outs/RegionLoupe.csv"
region_df = pd.read_csv(region_csv, index_col=0)
region_df.index = region_df.index + "_1"
print(region_df)
common_barcodes = rna_norm.index
region_df = region_df.loc[common_barcodes]
print(region_df)
region_df.index.name = 'barcode'
coords_df.index.name = 'barcode'
coords_small = coords_df[['x', 'y']]
df_merged = region_df.join(coords_small, how='inner')
print(df_merged.head())
deconv = pd.read_csv("../deconvolution.tsv", sep="\t", index_col=0)
df_final  = df_merged.join(deconv, how="inner")
df_final  = df_final.reset_index()
df_final = df_final.set_index("index")
cell_types = [c for c in df_final.columns if c not in ['RegionLoupe','x','y']]
prop_df = df_final[cell_types]
print(prop_df)

                     RegionLoupe
Barcode                         
AAACAAGTATCTCCCA-1_1          CP
AAACAGCTTTCAGAAG-1_1         CTX
AAACAGGGTCTATATT-1_1         NaN
AAACATTTCCCGGATT-1_1          cc
AAACCCGAACGAAATC-1_1          CP
...                          ...
TTGTTCAGTGTGCTAC-1_1         ACB
TTGTTCTAGATACGCT-1_1         CTX
TTGTTGTGTGTCAAGA-1_1         PAL
TTGTTTCCATACAACT-1_1         CTX
TTGTTTGTGTAAATTC-1_1         NaN

[2856 rows x 1 columns]
                     RegionLoupe
TGGGCACAAACAGAAC-1_1         NaN
TTGGAGTCTCCCTTCT-1_1         NaN
AATATCAAGGTCGGAT-1_1         CTX
CACCCTTTCCTCGCTC-1_1         CTX
CTGGTTCAACGCATCA-1_1         CTX
...                          ...
GCTCAACCTCTTAGAG-1_1         CTX
GGGCGAATTTCTCCAC-1_1         CTX
TGAAGTAGCTTACGGA-1_1         CTX
TACATCTTGTTTCTTG-1_1         CTX
CAGTAGATGATGTCCG-1_1         CTX

[2386 rows x 1 columns]
                     RegionLoupe   x  y
barcode                                
TGGGCACAAACAGAAC-1_1         NaN  46  8
TTGGA

In [6]:
ROOT = Path("../results_by_metabolite_l4_n_es1")
PREFERRED_NAME = "msi_norm_hvg_162_shuffled.csv"
GLOB_FALLBACK = "*msi*norm*hvg*162*shuffled*.csv"
mz_dirs = sorted(
    [p for p in ROOT.iterdir() if p.is_dir() and (p / "metabolite").is_dir()],
    key=lambda p: float(re.findall(r"[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?", p.name)[0])
)

frames = []
missing = []

for d in mz_dirs:
    mdir = d / "metabolite"
    f = mdir / PREFERRED_NAME
    if not f.exists():
        cands = list(mdir.glob(GLOB_FALLBACK))
        if not cands:
            missing.append(str(d))
            continue
        f = sorted(cands, key=lambda x: len(x.name))[0]

    df = pd.read_csv(f)
    idx_col = None
    for c in df.columns[:2]:
        if str(c).lower() in ("spot", "spots", "index", "unnamed: 0"):
            idx_col = c
            break
    if idx_col is not None:
        df = df.set_index(idx_col)

    val_col = df.columns[0]
    ser = df[val_col].copy()
    ser.name = d.name

    frames.append(ser)

if not frames:
    raise RuntimeError("Cannot find shuffled CSV")

msi_null_hvg = pd.concat(frames, axis=1)

out_path = "msi_null_hvg.csv"
msi_null_hvg.to_csv(out_path)

print(f"Saved: {out_path}")
if missing:
    print(f"{len(missing)} metabolites missing shuffled CSV, examples:")
    print("\n".join(missing[:10]))


Saved: msi_null_hvg.csv


In [8]:
decimals_try = [4, 5, 6, 7]
tol = 1e-7
use_rank_for_nonexact = True
save_prefix = "msi_null_from_round"

def detect_exact_by_decimals(msi_real, msi_null, decimals_try=[4,5], tol=1e-8):
    cols = msi_real.columns.intersection(msi_null.columns)
    rows = msi_real.index.intersection(msi_null.index)
    reports = []
    for c in cols:
        a = msi_real.loc[rows, c].to_numpy()
        b = msi_null.loc[rows, c].to_numpy()
        if np.isnan(a).sum() != np.isnan(b).sum():
            reports.append((c, "no_exact", np.nan, False, "NaN_count_diff"))
            continue
        a_f = a[np.isfinite(a)]
        b_f = b[np.isfinite(b)]
        if a_f.size != b_f.size:
            reports.append((c, "no_exact", np.nan, False, "finite_len_diff"))
            continue

        found = False
        found_d = None
        for d in decimals_try:
            a_r = np.round(a_f, d)
            if np.allclose(np.sort(a_r, kind="mergesort"),
                           np.sort(b_f, kind="mergesort"),
                           atol=tol, rtol=0.0):
                found = True
                found_d = int(d)
                break
        if found:
            reports.append((c, "exact_permutation", found_d, True, "ok"))
        else:
            if a_f.size > 0:
                mad = float(np.max(np.abs(np.sort(a_f) - np.sort(b_f))))
            else:
                mad = 0.0
            reports.append((c, "no_exact", np.nan, False, f"values_diff_maxabs={mad:.3g}"))
    rep_df = pd.DataFrame(reports, columns=["metabolite","method","decimals","ok","reason"]).set_index("metabolite")
    return rep_df

report = detect_exact_by_decimals(msi_norm_hvg, msi_null_hvg, decimals_try=decimals_try, tol=tol)
print(report["method"].value_counts())
# report.to_csv(f"{save_prefix}_detection_report.csv")

def apply_mappings_with_round(msi_real, msi_null, msi_target, report_df, use_rank_for_nonexact=True):
    rows_common = msi_real.index.intersection(msi_null.index)
    cols = report_df.index.intersection(msi_target.columns)
    out = msi_target.copy()
    mapping_info = []

    for c in cols:
        row = report_df.loc[c]
        method = row["method"]
        if method == "exact_permutation" and not pd.isna(row["decimals"]):
            d = int(row["decimals"])
            a = msi_real.loc[rows_common, c].to_numpy()
            b = msi_null.loc[rows_common, c].to_numpy()
            finite_mask = np.isfinite(a) & np.isfinite(b)
            if finite_mask.sum() == 0:
                mapping_info.append((c, "exact_permutation", d, 0))
                continue
            idx = np.array(rows_common)[finite_mask]
            a_f = np.round(a[finite_mask], d)
            b_f = b[finite_mask]
            order_a = np.argsort(a_f, kind="mergesort")
            order_b = np.argsort(b_f, kind="mergesort")
            perm_pos = np.empty(len(idx), dtype=int)
            perm_pos[order_b] = order_a
            src_labels = idx[perm_pos]
            dest_labels = idx
            mapping_series = pd.Series(src_labels, index=dest_labels)
            dest_present = out.index.intersection(mapping_series.index)
            if len(dest_present) > 0:
                src_for_dest = mapping_series.loc[dest_present].values
                mask_src_in_out = pd.Index(src_for_dest).isin(out.index)
                if not mask_src_in_out.all():
                    keep_idx = np.where(mask_src_in_out)[0]
                    dest_present = dest_present[keep_idx]
                    src_for_dest = src_for_dest[keep_idx]
                out.loc[dest_present, c] = out.loc[src_for_dest, c].to_numpy()
            mapping_info.append((c, "exact_permutation", d, len(dest_present)))
        else:
            if not use_rank_for_nonexact:
                mapping_info.append((c, "unchanged", np.nan, 0))
                continue
            # rank_map fallback
            a = msi_real.loc[rows_common, c].to_numpy()
            b = msi_null.loc[rows_common, c].to_numpy()
            finite_mask = np.isfinite(a) & np.isfinite(b)
            if finite_mask.sum() == 0:
                mapping_info.append((c, "rank_map", np.nan, 0))
                continue
            idx = np.array(rows_common)[finite_mask]
            a_f = a[finite_mask]; b_f = b[finite_mask]
            order_a = np.argsort(a_f, kind="mergesort")
            order_b = np.argsort(b_f, kind="mergesort")
            perm_pos = np.empty(len(idx), dtype=int)
            perm_pos[order_b] = order_a
            src_labels = idx[perm_pos]; dest_labels = idx
            mapping_series = pd.Series(src_labels, index=dest_labels)
            dest_present = out.index.intersection(mapping_series.index)
            if len(dest_present) > 0:
                src_for_dest = mapping_series.loc[dest_present].values
                mask_src_in_out = pd.Index(src_for_dest).isin(out.index)
                if not mask_src_in_out.all():
                    keep_idx = np.where(mask_src_in_out)[0]
                    dest_present = dest_present[keep_idx]
                    src_for_dest = src_for_dest[keep_idx]
                out.loc[dest_present, c] = out.loc[src_for_dest, c].to_numpy()
            mapping_info.append((c, "rank_map", np.nan, len(dest_present)))

    maprep = pd.DataFrame(mapping_info, columns=["metabolite","applied_method","decimals_used","n_rows_mapped"]).set_index("metabolite")
    return out, maprep

msi_df_null, maprep = apply_mappings_with_round(
    msi_norm_hvg, msi_null_hvg, msi_df_hvg, report,
    use_rank_for_nonexact=use_rank_for_nonexact
)

# report.to_csv(f"{save_prefix}_detection_report.csv")
# maprep.to_csv(f"{save_prefix}_applied_map_report.csv")
msi_df_null.to_csv("msi_df_null.csv")

print(" - generated null data:", f"{save_prefix}_msi_df_null.csv")
print("\nSummary of applied methods (on columns present in target):")
print(maprep["applied_method"].value_counts())

method
no_exact             366
exact_permutation    134
Name: count, dtype: int64
 - generated null data: msi_null_from_round_msi_df_null.csv

Summary of applied methods (on columns present in target):
applied_method
rank_map             366
exact_permutation    134
Name: count, dtype: int64
